In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

chef_prompt = ChatPromptTemplate.from_messages([
    ('system', 'you are a world class international chef. you create easy to follow recipies for any type of cuisine with easy to fin dingredients.'),
    ('human', 'I want to cook {cuisine} food.'),
])

chef_chain = chef_prompt | chat

In [6]:
veg_chef_prompt = ChatPromptTemplate.from_messages([
('system', 'you are a vegetarian chef specialized on making traditional recipies vegetarian. you find alternative ingredients and explain their preparation. You do not radically modify the recipe. If there is no alternative for a food, just say you do not know how to replace it'),
('human', '{recipe}'),
])

veg_chain = veg_chef_prompt | chat

final_chain = {"recipe" :chef_chain} | veg_chain

final_chain.invoke({"cuisine": "indian"})

Great choice! Indian cuisine is known for its rich flavors and aromatic spices. Here's a simple recipe for Chicken Tikka Masala, a popular Indian dish:

Ingredients:
- 500g boneless chicken, cut into bite-sized pieces
- 1 cup plain yogurt
- 2 tablespoons lemon juice
- 2 tablespoons vegetable oil
- 1 large onion, finely chopped
- 3 cloves of garlic, minced
- 1-inch piece of ginger, grated
- 2 teaspoons ground cumin
- 2 teaspoons ground coriander
- 1 teaspoon turmeric powder
- 1 teaspoon paprika
- 1 teaspoon garam masala
- 1 cup tomato puree
- 1 cup heavy cream
- Salt, to taste
- Fresh cilantro, chopped (for garnish)

Instructions:
1. In a bowl, combine the yogurt, lemon juice, 1 teaspoon of cumin, 1 teaspoon of coriander, turmeric powder, paprika, and salt. Mix well.
2. Add the chicken pieces to the marinade, ensuring they are well coated. Cover and refrigerate for at least 1 hour, or overnight for best results.
3. Preheat your oven to 400°F (200°C). Place the marinated chicken on a bak

AIMessageChunk(content="Great choice! Chicken Tikka Masala is a delicious dish, and I can help you make a vegetarian version of it. Instead of using chicken, we can substitute it with a plant-based protein like tofu or paneer. Here's how you can modify the recipe:\n\nIngredients:\n- 500g tofu or paneer, cut into bite-sized pieces\n- 1 cup plain yogurt (use dairy-free yogurt if vegan)\n- 2 tablespoons lemon juice\n- 2 tablespoons vegetable oil\n- 1 large onion, finely chopped\n- 3 cloves of garlic, minced\n- 1-inch piece of ginger, grated\n- 2 teaspoons ground cumin\n- 2 teaspoons ground coriander\n- 1 teaspoon turmeric powder\n- 1 teaspoon paprika\n- 1 teaspoon garam masala\n- 1 cup tomato puree\n- 1 cup coconut cream (or dairy-free heavy cream if vegan)\n- Salt, to taste\n- Fresh cilantro, chopped (for garnish)\n\nInstructions:\n1. In a bowl, combine the yogurt, lemon juice, 1 teaspoon of cumin, 1 teaspoon of coriander, turmeric powder, paprika, and salt. Mix well.\n2. Add the tofu or